In [0]:
import os
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Activation, Flatten, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Reshape, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.models import Model

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [0]:
current_path = '/content/drive/My Drive/Colab Notebooks/digit-recognizer'
# パス移動
os.chdir(current_path)

In [0]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
print(train.shape)
print(test.shape)

(42000, 785)
(28000, 784)


In [0]:
d1 = 28
d2 = 28
imageSize = d1*d2
vc_rate = 0.9

In [0]:
# ラベル(OneHot)
labels = pd.get_dummies(train['label'], prefix='label')
train_data = train.drop('label', axis=1)
print(labels.shape)
print(train_data.shape)

(42000, 10)
(42000, 784)


In [0]:
train_data = np.array(train_data)
test_data = np.array(test)

In [0]:
train_data = train_data.reshape([len(train), d1, d2, 1])
test_data = test_data.reshape([len(test), d1, d2, 1])
print(train_data.shape)
print(test_data.shape)

(42000, 28, 28, 1)
(28000, 28, 28, 1)


In [0]:
# 正規化
train_data = train_data / 255
test_data = test_data / 255

In [0]:
def create_bench_model():
    inputs = Input(shape = (d1,d2,1))
    x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(inputs)
    x = Conv2D(64,(3,3),padding = "SAME",activation= "relu")(x)
    x = Dropout(0.25)(x)
    x = MaxPooling2D()(x)

    x = Conv2D(128,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(128,(3,3),padding = "SAME",activation= "relu")(x)
    x = Dropout(0.25)(x)
    x = MaxPooling2D()(x)

    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = Conv2D(256,(3,3),padding = "SAME",activation= "relu")(x)
    x = GlobalAveragePooling2D()(x)

    x = Dense(1024,activation = "relu")(x)
    x = Dropout(0.25)(x)
    y = Dense(10,activation = "softmax")(x)

    return Model(input = inputs, output = y)

In [0]:
def my_eval(model,x,t):
    ev = model.evaluate(x,t)
    print("loss:" ,end = " ")
    print(ev[0])
    print("acc: ", end = "")
    print(ev[1])

In [0]:
model = create_bench_model()
model.compile('adam', "categorical_crossentropy", metrics = ["accuracy"])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
es = EarlyStopping(monitor='val_acc')
model.fit(train_data, labels, nb_epoch=300, validation_split=0.2, callbacks=[es])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  





Train on 33600 samples, validate on 8400 samples
Epoch 1/300





33600/33600 [==============================] - 16s 487us/step - loss: 0.4466 - acc: 0.8434 - val_loss: 0.1087 - val_acc: 0.9756
Epoch 2/300
33600/33600 [==============================] - 13s 372us/step - loss: 0.0638 - acc: 0.9810 - val_loss: 0.0776 - val_acc: 0.9781
Epoch 3/300
33600/33600 [==============================] - 13s 375us/step - loss: 0.0479 - acc: 0.9854 - val_loss: 0.0458 - val_acc: 0.9865
Epoch 4/300
33600/33600 [==============================] - 12s 367us/step - loss: 0.0400 - acc: 0.9879 - val_loss: 0.0426 - val_acc: 0.9876
Epoch 5/300
33600/33600 [==============================] - 12s 369us/step - loss: 0.0355 - acc: 0.9895 - val_loss: 0.0333 - val_acc: 0.9912
Epoch 6/300
33600/33600 [==============================] - 12s 369us/step - loss: 0.0303 - acc: 0.9907 - val_loss: 0.0452 - val_acc: 0.9882


In [0]:
my_eval(model, train_data, labels)

42000/42000 [==============================] - 4s 94us/step
loss: 0.028705918151840922
acc: 0.9918095238095238


In [0]:
pred_y = model.predict(test_data)
pred_y.shape

(28000, 10)

In [0]:
# dummy -> cat
cat_y = np.argmax(pred_y, axis=1)
print(cat_y[0])
print(len(cat_y))

2
28000


In [0]:
# 結果出力
predict = pd.DataFrame({'ImageId': range(1, 28001), 'Label':cat_y})
print(predict.shape)
print(predict.head())

predict.to_csv('tensorflow_predict.csv', index=False)

(28000, 2)
   ImageId  Label
0        1      2
1        2      0
2        3      9
3        4      0
4        5      3
